# Обработка данных по продажам бизнес единицы Т

### Библиотеки

In [1]:
# основные библиотеки
import pandas as pd
import numpy as np

In [2]:
# вспомогательные библиотеки
from datetime import datetime
import time
import pyodbc
from tqdm import tqdm

In [3]:
# этот волшебный код убирает ненужные предупреждения об ошибках
import warnings
warnings.filterwarnings('ignore')

In [4]:
#настройка, чтобы можно было просматривать все столбцы датафрейма
import matplotlib
matplotlib.rcParams.update({'font.size':14})
pd.options.display.max_columns=100

### Методы

In [5]:
def load_fact(PATH_FILE):
    
    '''Загрузка данных по продажам, обработанных в Power BI'''
    # Загружаем файл, обработанный в Power BI
    df_fact = pd.read_csv(PATH_FILE)
    return df_fact

In [6]:
def load_klass_nom_bd():

    '''Загрузка классификатора номенклатуры'''
    # задаём параметры соединения с базой данных
    cn = pyodbc.connect("Driver={SQL Server Native Client 11.0};\
                        Server=s;\
                        Database=analitics;\
                        Trusted_Connection=yes")
    
    # скачиваем из базы классификатор номенклатуры
    df_klass_nom_bd = pd.read_sql_query('SELECT [КодНоменклатуры]\
        ,[Артикулt]\
        ,[Наименование]\
        ,[Группа_1]\
        ,[Группа_2]\
        ,[Группа_3]\
        ,[Собственное_производство]\
    FROM [Analitics].[dbo].[КлассификаторНоменклатуры]', cn)
    
    return df_klass_nom_bd

In [7]:
def load_expense():
    '''Загрузка данных по накладным расходам'''
    df_expense = pd.read_excel('C:/Users/lazarevnv/Desktop/Инфа по подключению t в mdm/Выгрузки_1С_t/Накладные_расходы_ФД.xlsx')
    return df_expense

In [20]:
def load_all(PATH_FILE):

    'Загружаем сразу все нужные таблицы'
    df_fact = load_fact(PATH_FILE)
    df_klass_nom_bd = load_klass_nom_bd()
    df_expense = load_expense()

    return df_fact, df_klass_nom_bd, df_expense

In [26]:
def first_show(df_fact):
    '''Метод обрбабатывает данные после BI'''
    # смотрим пример того, что загрузили 
    print('смотрим пример того, что загрузили')
    display(df_fact.tail(2))
    
    # смотрим размеры таблицы
    print('смотрим размеры таблицы', df_fact.shape)

    # смотрим называния колонок
    print('смотрим называния колонок', df_fact.columns)

    # смотрим огрегированные данные
    print('смотрим огрегированные данные')
    display(df_fact[['Количество', 'Продажи_сНДСРуб', 'Продажи_безНДСРуб', 'Валовая прибыль (RUB)']].sum())

    # удалим поле Мера для подсчёта строк
    df_fact = df_fact.drop(columns=['Мера_для подсчёта строк'])

    #  удалим строки с пустым значением Себестоимость  (RUB) 
    #  т.к. это Лом латуни
    df_fact = df_fact.dropna(subset = ['Артикулt', 'Себестоимость  (RUB)']).reset_index(drop=True)
    
    # отлавливаем пропуски
    print('отлавливаем пропуски')
    display(df_fact.isna().sum())

    # Переименуем поля, дадим более правильные названия с точки зрения ФД
    df_fact = df_fact.rename(columns={'Валовая прибыль (RUB)':'Валовая_прибыль_t',\
                 'Себестоимость  (RUB)':'Прямые_расходы(СиМ)', 'Накладные расходы':'Общие_наклад_расх_мес'})
    
    print('Результат функции first_show')
    display(df_fact.tail(2))
    
    return df_fact

In [10]:
def breakdown_rf_sz(df_fact):

     '''Разбивка на t-Россия и t-СЗ'''

     # добавляем пустое поле
     df_fact['ТорговаяПлощадка'] = 't-Россия'

     # присваиваем полю ТорговаяПлощадка значение t-СЗ по условию
     df_fact.loc[df_fact['Регион'].isin(['Узбекистан', 'Казахстан', 'Белоруссия']) ,\
          'ТорговаяПлощадка'] = 't-СЗ'
     print('Результат функции breakdown_rf_sz')
     display(df_fact.tail(2))
     return df_fact

In [11]:
def processing_expense(df_fact, df_expense):

    '''Расчёт актуальной валовой прибыли'''

    # добавляем в факт столбец с первым днём месяца
    df_fact['Месяц_'] = pd.to_datetime(df_fact['Месяц'])

    # добавляем поле с первым числом месяца
    df_fact['Месяц_'] = (df_fact['Месяц_'].dt.floor('d') \
        + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))

    # добавляем поле с данными из таблицы df_expense
    df_fact = df_fact.merge(df_expense, left_on=['Месяц_'], \
         right_on=['Дата'], how='left')
    # Переименуем поля, дадим более правильные названия с точки зрения ФД
    df_fact = df_fact.rename(columns={'Валовая прибыль (RUB)':'Валовая_прибыль_t',\
         'Себестоимость  (RUB)':'Прямые_расходы(СиМ)', 'Накладные расходы':'Общие_наклад_расх_мес'})

    # добавляем признак Собственное_производство
    df_fact = df_fact.merge(df_klass_nom_bd[['Артикулt','Собственное_производство']],\
        on='Артикулt', how='left')
    
    # принимаем допущение, что если артикула нет в классификаторе номенклатуты, то ставим 100
    df_fact['Собственное_производство'] = df_fact['Собственное_производство'].fillna(100)

    # указанные гофрокороба, это сырьё, поэтому нужно убрать их из продаж
    # сформируем список артикулов по этим гофрокоробам
    material_list = df_fact.loc[(df_fact['Собственное_производство']==100) 
            &~(
                    (df_fact['Номенклатура'].str.contains('Лом'))
                    |(df_fact['Номенклатура'].str.contains('лом'))
                    |(df_fact['Номенклатура'].str.contains('НЕ ИСПОЛЬЗОВАТЬ'))
                ), 'Артикулt'].to_list()
    
    # сформируем условия для удаления ненужных строк
    conditions_for_delete = (
        (df_fact['Номенклатура'].str.contains('Лом'))
        |(df_fact['Номенклатура'].str.contains('лом'))
        |(df_fact['Номенклатура'].str.contains('НЕ ИСПОЛЬЗОВАТЬ'))
        |(df_fact['Артикулt'].isin(material_list)) )

    # формируем перечень позиций которых нет в классификаторе. Ищем ошибки
    print('Проверка. Формируем перечень позиций которых нет в классификаторе. Если ошибок нет, то датасет пуст.')
    display(df_fact.loc[(df_fact['Собственное_производство']==100) 
            &~(conditions_for_delete)])

    # убираем из файла продажи по условию conditions_for_delete (лом и Не использовать)
    # формируем новый датафрейм, очищенный от лишних строк
    df_fact_clean = df_fact.loc[~conditions_for_delete]

    print(f'Смотрим габариты df_fact {df_fact.shape}, и df_fact_clean {df_fact_clean.shape}')

    # посчитаем ежемесячные прямые расходы(СиМ), но только для продукции собственного производства!
    df_direct_expenses = \
    df_fact_clean.loc[df_fact_clean['Собственное_производство']==1]\
    .pivot_table(index ='Месяц_'
                ,columns=None 
                ,values=['Прямые_расходы(СиМ)']
                ,aggfunc=sum)
    print('Посчитаем ежемесячные прямые расходы(СиМ), но только для продукции собственного производства!')
    display(df_direct_expenses)
    # переименуем поле
    df_direct_expenses = df_direct_expenses.rename(columns={'Прямые_расходы(СиМ)':'Общие_прямые_расх_CиМ_мес'})

    # подтягиваем нужные данные
    df_fact_clean = df_fact_clean.merge(df_direct_expenses, on='Месяц_', how='left')

    # Присваиваем полю Общие_прямые_расх_CиМ_мес значение None для аутсорсинга
    df_fact_clean.loc[df_fact_clean['Собственное_производство']!=1,
                'Общие_прямые_расх_CиМ_мес'] = None

    # Рассчитываем накладные расходы для каждой строки
    # Создадим новое расчётное поле Накладные_расходы
    df_fact_clean['Накладные_расходы'] = df_fact_clean['Прямые_расходы(СиМ)']\
        / df_fact_clean['Общие_прямые_расх_CиМ_мес']\
        * df_fact_clean['Общие_наклад_расх_мес']

    # зафиксируем Накладные_расходы для аутсорсинга
    df_fact_clean.loc[df_fact_clean['Собственное_производство']==0, 'Накладные_расходы'] = 0

    # зафиксируем Накладные_расходы для товаров отсутствующих в классификаторе
    df_fact_clean.loc[df_fact_clean['Собственное_производство']==100, 'Накладные_расходы'] = 0

    # Создадим новое расчётное поле Себестоимость
    df_fact_clean['Себестоимость'] = df_fact_clean['Прямые_расходы(СиМ)'] + df_fact_clean['Накладные_расходы']

    # Создадим новое расчётное поле Валовая_прибыль
    df_fact_clean['Валовая_прибыль'] = df_fact_clean['Продажи_безНДСРуб'] - df_fact_clean['Себестоимость']

    #  удалим лишнее поле Дата
    df_fact_clean = df_fact_clean.drop(columns=['Дата'])

    # Переименуем поле Месяц в Дата
    df_fact_clean = df_fact_clean.rename(columns={'Месяц':'Дата'})
    
    # перед экспортом проверяем пустые значения
    print('перед экспортом проверяем пустые значения, всё должно быть по нулям')
    display(df_fact_clean[['Дата', 'Месяц_', 'КодНоменклатуры', 'Артикулt', 'Собственное_производство', 

    'Количество', 'Продажи_сНДСРуб', 'Продажи_безНДСРуб', 
    'Прямые_расходы(СиМ)', 'Накладные_расходы', 'Себестоимость', 'Валовая_прибыль',

    'Код_контрагента', 'Контрагент', 'Регион',  'ТорговаяПлощадка', 'Источник']].isna().sum())

    # смотрим, что получилось
    print('Смотрим, что получилось. Результат работы функции processing_expense')
    display(df_fact_clean.tail())

    return df_fact_clean

In [31]:
def export_fact_t(df):

    df = df[['Дата', 'Месяц_', 'КодНоменклатуры', 'Артикулt', 'Собственное_производство', 

    'Количество', 'Продажи_сНДСРуб', 'Продажи_безНДСРуб', 
    'Прямые_расходы(СиМ)', 'Накладные_расходы', 'Себестоимость', 'Валовая_прибыль',

    'Код_контрагента', 'Контрагент', 'Регион',  'ТорговаяПлощадка', 'Источник']]
    
    '''Метод экспортирует данные факт t 2023 в базу данных Analitics'''
    data_server = 's'
    database = 'Analitics'
    cn = pyodbc.connect("Driver={SQL Server Native Client 11.0};\
                        Server="+data_server+";Database="+database+";Trusted_Connection=yes")
    cursor = cn.cursor()
    for index, row in tqdm(df.iterrows()):
        cursor.execute(
            "INSERT INTO  dbo.[Продажи_t] (\
             [Дата], [Месяц_], [КодНоменклатуры], [Артикулt], [Собственное_производство] \
            ,[Количество], [Продажи_сНДСРуб], [Продажи_безНДСРуб] \
            ,[Прямые_расходы(СиМ)], [Накладные_расходы], [Себестоимость], [Валовая_прибыль]\
            ,[Код_контрагента], [Контрагент], [Регион], [ТорговаяПлощадка], [Источник] \
                                                ) \
            values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",

            row['Дата'], row['Месяц_'], row['КодНоменклатуры'], row['Артикулt'], row['Собственное_производство'] \
            ,row['Количество'], row['Продажи_сНДСРуб'], row['Продажи_безНДСРуб'] \
            ,row['Прямые_расходы(СиМ)'], row['Накладные_расходы'], row['Себестоимость'], row['Валовая_прибыль']\
            ,row['Код_контрагента'], row['Контрагент'], row['Регион'], row['ТорговаяПлощадка'], row['Источник']
        )

    cn.commit()
    cursor.close()
    cn.close()

### Запускаем обработку

In [ ]:
# Запускаем загрузку всех таблиц
file_name = 't_fact_2023_06&07_09.08.2023.csv'
PATH_FILE = f'C:/Users/lazarevnv/Desktop/Инфа по подключению t в mdm/Выгрузки_BI/{file_name}'

df_fact, df_klass_nom_bd, df_expense = load_all(PATH_FILE)

# запускаем предварительную обработку
df_fact = first_show(df_fact)

In [ ]:
# запускаем разбивку на рф и сз
df_fact = breakdown_rf_sz(df_fact)

In [ ]:
# запускаем функцию расчёта накладных расходов и валовой прибыли
df_fact_clean = processing_expense(df_fact, df_expense)

Перед заливкой данных нужно удалить из базы предыдущий месяц. Потом добавим сюда отдельный метод для удаления из python

In [32]:
%%time
# заливаем данные
export_fact_t(df_fact_clean)

5600it [00:44, 126.47it/s]

CPU times: total: 359 ms
Wall time: 44.5 s
